# Calibrate Steering

**This notebook is used to find the correspondance between the command steering and the real steering**

Assume $\delta_{\text{real}} = a \times \delta_\text{target} + b$

We collect data on cars driving in both clockwise and counterclockwise circles at a low speed of 3m/s using Vicon.

## Fetch data

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import utm
import scipy
import scipy.interpolate
import scipy.signal
from scipy.spatial.transform import Rotation
# from car_dynamics.envs.kinematic_bicycle_model import LF, LR
from car_dynamics.analysis import pos2vel_savgol, calc_delta, interpolate, rotate_point, calc_delta_v
from offroad.utils import load_state
import ipywidgets as widgets
from ipywidgets import interact
from termcolor import colored
%matplotlib inline
%load_ext autoreload
%autoreload 2
LF = .16
LR = .15
log_dir = '/Users/wenlixiao/Dropbox/School/Graduate/LeCAR/safe-learning-control/playground/offroad/data/dataset/' 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
t_list, p_dict, yaw_dict, action_list = load_state(log_dir + 'data-20240111-234024', [0,118] , orientation_provider="IENTATION_PROVIDOER")

ValueError: too many values to unpack (expected 4)

In [13]:
log_dir = '/Users/wenlixiao/Dropbox/School/Graduate/LeCAR/safe-learning-control/playground/offroad/data/dataset/' 

data_list = [
                # dict(name='straight', dir='data-20240105-131330', range=[0,3]),
                # dict(name='straight', dir='data-20240105-131642', range=[0,3]),
                # dict(name='straight', dir='data-20240105-131554', range=[0,3]),
                # dict(name='arc left', dir='data-20240105-132023', range=[0,8]),
                # dict(name='arc left', dir='data-20240105-132200', range=[0,8]),
                # dict(name='arc left', dir='data-20240105-132304', range=[0,9]),
                # dict(name='arc right', dir='data-20240105-132434', range=[0,13]),
                # dict(name='arc right', dir='data-20240105-132718', range=[0,20]),
                # dict(name='arc right', dir='data-20240105-133440', range=[0,13]),
                # dict(name='circle', dir='data-20240105-144318', range=[0,101]),
                # dict(name='circle', dir='data-20240105-151616', range=[0,55]),
                # dict(name='clock', dir='data-20240105-151616', range=[0,43]),
                # dict(name='clock', dir='data-20240105-151931', range=[0,38]),
                # dict(name='clock', dir='data-20240105-152139', range=[0,38]),
                # dict(name='clock', dir='data-20240105-152431', range=[0,38]),
                # dict(name='clock', dir='data-20240105-171153', range=[0,158]),
                dict(name='clock', dir='data-20240302-151638', range=[0,146]),
                dict(name='clock', dir='data-20240302-151849', range=[0,201]),
                # dict(name='clock', dir='data-20240105-160800', range=[0,90]),
                # dict(name='clock', dir='data-20240105-164406', range=[0,58]),
                # dict(name='clock', dir='data-20240105-172301', range=[0,61]),

    # dict(name='clock', dir='data-20231226-171157', range=[10, 136]),
                # dict(name='counter clock', dir='data-20231226-170639', range=[7, 213]),
                # dict(name='clock', dir='data-20231226-171421', range=[20, 110]),
                # dict(name='counter clock', dir='data-20231226-170909', range=[7, 82]),
                # dict(name='unknown', dir='data-20231227-185416', range=[0, 52]),
                # dict(name='unknown', dir='data-20231227-183524', range=[0, 161]),
            ]

In [14]:
result_list = []
for data in data_list:
    t_list, p_dict, yaw_dict, action_list, _ = load_state(log_dir + data['dir'], data['range'] , orientation_provider="IMU")
    yaw_vicon = yaw_dict['vicon']
    p_list = p_dict['vicon']
    
    ## Calibrate pos and yaw
    px, py = rotate_point(p_list[:, 0], p_list[:, 1], 90)
    p_rot = np.zeros((len(px),2))
    p_rot[:,0] = px
    p_rot[:,1] = py
    yaw_vicon_calibrated = yaw_vicon + np.pi/2
    yaw_vicon_calibrated = np.arctan2(np.sin(yaw_vicon_calibrated), np.cos(yaw_vicon_calibrated))

    yaw_smooth = [yaw_vicon_calibrated[0]]
    running_yaw = yaw_smooth[0]
    yaw_alpha = 1.0
    for yaw in yaw_vicon_calibrated[1:]:
        yaw_smooth.append(yaw_smooth[-1] + yaw_alpha * (yaw - yaw_smooth[-1]))
    # plt.figure()
    # plt.plot(yaw_smooth, label='smooth')
    # plt.plot(yaw_vicon_calibrated, label='raw')
    # plt.legend()
    dt = 0.05
    p_smooth, vel, vel_vec = pos2vel_savgol(p_rot,window_length=5, delta=dt)
    # delta, beta = calc_delta_v(vel_vec, yaw_vicon_calibrated, LF, LR)
    delta, beta = calc_delta_v(vel_vec, yaw_smooth, LF, LR)
    result_list.append(
        dict(name=data['name'], action=action_list.copy(), delta=delta.copy())
    )
def plot(a=.46, b=-0.13, delay=4):
    fig, axs = plt.subplots(1, len(result_list), figsize=(10*len(result_list), 5))
    for i, result in enumerate(result_list):
        axs[i].plot(result['delta'], label='real delta')
        proj_action = result['action'][:, 1]*a+b
        delayed_action = np.zeros_like(proj_action)
        delayed_action[delay:] = proj_action[:len(proj_action)-delay]
        axs[i].plot(delayed_action, label='target delta')
        axs[i].legend()
        axs[i].set_title(f"{result['name']}")

In [15]:
interact(plot, 
         a = (-1, 1, 0.01),
         b = (-1, 1, 0.01),
         delay=(0, 100, 1),
        )

interactive(children=(FloatSlider(value=0.46, description='a', max=1.0, min=-1.0, step=0.01), FloatSlider(valu…

<function __main__.plot(a=0.46, b=-0.13, delay=4)>